This is my primary notebook for my data analyst stuff for now. 

To-do now:
- Write data collection function(s)
- Port non-analysis code (such as data collection) to a normal python file in vscode or sum

To-do later: 
- Ask opinions from players/coaches
- Begin data analysis
- Perhaps port to sql server/cloud rather than sqlite local
- Find other players to add from other teams

Stretch goals:
- Drafting automation
- Front-end

In [1]:
# dependency list. 

import random
import cassiopeia as cass
from cassiopeia import Account, Queue, Summoner, Match
import pandas as pd
import sqlite3
from datetime import datetime
import pytz

# This option makes sure that pandas doesn't display large numbers (such as IDs) in scientific notation
pd.set_option('display.float_format', lambda x: '%.0f' % x)

# This option makes sure that pandas doesn't cut off long fields (like item lists) with a "..."
pd.set_option('display.max_colwidth', None)

# This fixes shit from breaking when Riot (small indie company) is too slow to update stuff when they drop new patches
cass.apply_settings({
    "global": {
        "version_from_match": "latest",
    }
})

In [2]:
# this needs to be run to reset the api key. key decays every day. if you're getting request denied errors it's prob cause this decayed
cass.set_riot_api_key("RGAPI-433a2a7d-760a-418b-8816-d5c34b912898")

In [3]:
ids = ['VHX5slvZYjyVJUw1fSJakRsqvGyKuaeH3dpjb-BwRDk1x0M', 
       'B00G4_ZFmXqfbKgD2kdL9eVGRwv0crCu1np4WFSS7Xz-pBybezDWmGTVaA', 
       'KIG0_-pbjEfZMjT2lb9yOdtvqjQ42mDubp4gVbvUpBLXbf0pSLdf-plzSw', 
       'mhk15HBYt1OiAvrctDW1bPVIf2_9AFHbXEonxDDXcVE2jjRPxaJQ5WufTg',
       't3n5_faML1XnN_wdINN4iT9aynSEzE-xVCdP0keK8y4t_6Y',
       'PbF4IGHY9xvMZ1InkxyWBx5lQ-S_E2bF0whqw9nGlT8HsCk',
       'dds5IGf4LqDbl4m8becDMAqlMH4q6Pj5uJQIVOqkkPtCefE'
      ]

In [4]:
# I AM GOING TO REWORK THIS

# it is not entirely required to have separate variables. could refer to it as player_ids[1] or something. 
# mostly for transparency and ease of changing and in case we need to refer to them easily down the line
top_id = ("Kevyi", "NA1")
jg_id = ("Evanlai", "goma")
mid_id = ("braindeadseagull", "cgull")
bot_id = ("stinky", "joker")
supp_id = ("Mach", "1234") 
player_ids = [top_id, jg_id, mid_id, bot_id, supp_id]

# same goes here, could easily just iterate through player_ids and stuff the summoner stuff into the players list
top = cass.get_account(name = top_id[0]  , tagline = top_id[1] , region = "NA").summoner
jg = cass.get_account(name = jg_id[0]    , tagline = jg_id[1]  , region = "NA").summoner
mid = cass.get_account(name = mid_id[0]  , tagline = mid_id[1] , region = "NA").summoner
bot = cass.get_account(name = bot_id[0]  , tagline = bot_id[1] , region = "NA").summoner
supp = cass.get_account(name = supp_id[0], tagline = supp_id[1], region = "NA").summoner

players = [top, jg, mid, bot, supp]
print(cass.get_account(name = "whats9plus10ha21", tagline = "MSM", region = "NA").puuid)
print(top.account)

print("done")

Making call: https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/Kevyi/NA1
Making call: https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/Evanlai/goma
Making call: https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/braindeadseagull/cgull
Making call: https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/stinky/joker
Making call: https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/Mach/1234
Making call: https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/whats9plus10ha21/MSM
30YT1Yv7QhDxvCLO7veAbGItBh_2g2ZXQhSNahxKMzkVe7Df__SD6qwoM3RHjvg0if2m9XNfPFoAxg
Account(puuid=LEDeKpP1WKnBFdkoITs_sCgvIJ3hVj3K5Xzh9J_zAcA3rNMXipEt8B0PH7E9ajgv0DtzgGj948eV4Q, name=?, tagline='?')
done


In [28]:
me = cass.get_account(name = "KC TARGHAMAS", tagline = "miku", region = "NA").summoner
me.level

Making call: https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/KC%20TARGHAMAS/miku


273

In [5]:
def print_rank(summoner):
    flex_tier = None
    flex_division = None
    flex_lp = None
    soloq_tier = None
    soloq_division = None
    soloq_lp = None
    
    league_entries = summoner.league_entries
    for entry in league_entries: 
        if entry.queue == Queue.ranked_flex_fives:
            print("flex: " + str(entry.tier) + " " + str(entry.division) + ", " + str(entry.league_points) + " lp")
        if entry.queue == Queue.ranked_solo_fives: 
            print("soloq: " + str(entry.tier) + " " + str(entry.division) + ", " + str(entry.league_points) + " lp")

def print_epics(team):
    barons = 0
    drakes = 0
    for player in team.participants:
        barons = barons + player.stats.baron_kills
        drakes = drakes + player.stats.dragon_kills
        
    print("barons: " + str(barons))
    print("heralds: " + str(team.rift_herald_kills))
    print("drakes: " + str(drakes))

def print_bans(team):
    print("bans: ", end=" ")
    for ban in team.bans:
        if ban is None: print("None", end=", ")
        else: print(ban.name, end=", ")
    print()

def print_runes(participant):
    for rune in participant.runes:
        print(rune.name, end=", ")
    for stat in participant.stat_runes:
        print(stat.name, end=", ")
    print()

def print_items(items):
    for item in items:
        if item is not None:
            print(item.name, end=", ")
        else:
            print("NULL", end=", ")
    print()
    
account = bot.account
match = bot.match_history[0]
team = match.teams[0]
participant = team.participants[3]

print("account:", end=" ")
print(account.puuid)
print(account.name + "#" + account.tagline + ", level " + str(bot.level))
print_rank(bot)
print()

print("match:", end=" ")
print(match.id)
print("date: " + datetime.fromisoformat(str(match.start)).strftime('%Y-%m-%d %H:%M:%S'))
print("patch: " + str(match.patch))
print("queue: " + str(match.queue))
print("duration: " + str(match.duration))
print("is remake: " + str(match.is_remake))
print()

print("team:", end=" ")
print(match.id)
print(team.side)
print("win: " + str(team.win))
print_bans(team)
print("towers: " + str(team.tower_kills))
print("inhibs: " + str(team.inhibitor_kills))
print_epics(team)
print()

print("participant:")
print("id: " + str(participant.id))
print("puuid: " + str(account.puuid))
print("match id: " + str(match.id))
print("account: " + account.name + "#" + account.tagline)
print("side: " + str(participant.side))
print("lane: " + str(participant.lane))
print("champion: " + participant.champion.name)
print_runes(participant)
print("summs: " + participant.summoner_spell_d.name + " (" + str(participant.stats.summoner_spell_1_casts) + " uses)" + ", " + participant.summoner_spell_f.name + " (" + str(participant.stats.summoner_spell_1_casts) + " uses)")
print("win: " + str(team.win))
print(str(participant.stats.kills) + "/" + str(participant.stats.deaths) + "/" + str(participant.stats.assists))
print("level: " + str(participant.stats.level))
print("gold: " + str(participant.stats.gold_earned))
print("cs: " + str(participant.stats.total_minions_killed))
print("jg monsters: " + str(participant.stats.neutral_minions_killed))
print_items(participant.stats.items)
print("dmg dealt: " + str(participant.stats.total_damage_dealt))
print("dmg dealt to champs: " + str(participant.stats.total_damage_dealt_to_champions))
print("dmg taken: " + str(participant.stats.total_damage_taken))
print("dmg mitigated: " + str(participant.stats.damage_self_mitigated))
print("inhib kills: " + str(participant.stats.inhibitor_kills))
print("inhib takedowns: " + str(participant.stats.inhibitor_takedowns))
print("turret kills: " + str(participant.stats.turret_kills))
print("turret takedowns: " + str(participant.stats.turret_takedowns))
print("vision: " + str(participant.stats.vision_score))
print("wards: " + str(participant.stats.wards_placed))
print("vision wards: " + str(participant.stats.vision_wards_placed))
print("ward kills: " + str(participant.stats.wards_killed))


# enter user id
# go through match list
# add match data
# add team data for both teams
# add participant data for each participant


Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/F04AYaQbkucx-uiioSmBWVAM3iGXRZBTUukTCnI1YyiOEVD_pvZUjsJNaS2sysBnPLAtuvFET_oORQ/ids?start=0&count=100


ValueError: not enough values to unpack (expected 2, got 1)

In [29]:
def print_table_schema(db_path, table_name):
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        for column in columns:
            print(column)
    print()

# Example usage
db_path = 'lol_data.db'
print_table_schema(db_path, 'Accounts')
print_table_schema(db_path, 'Matches')
print_table_schema(db_path, 'Teams')
print_table_schema(db_path, 'Participants')

(0, 'account_id', 'TEXT', 0, None, 1)
(1, 'summoner_name', 'TEXT', 1, None, 0)
(2, 'summoner_tagline', 'TEXT', 1, None, 0)
(3, 'summoner_level', 'INTEGER', 1, None, 0)
(4, 'summoner_flex_tier', 'TEXT', 0, None, 0)
(5, 'summoner_flex_division', 'TEXT', 0, None, 0)
(6, 'summoner_flex_lp', 'INTEGER', 0, None, 0)
(7, 'summoner_soloq_tier', 'TEXT', 0, None, 0)
(8, 'summoner_soloq_division', 'TEXT', 0, None, 0)
(9, 'summoner_soloq_lp', 'INTEGER', 0, None, 0)

(0, 'match_id', 'TEXT', 0, None, 1)
(1, 'patch', 'TEXT', 1, None, 0)
(2, 'date', 'TEXT', 1, None, 0)
(3, 'queue', 'TEXT', 1, None, 0)
(4, 'duration', 'TEXT', 1, None, 0)
(5, 'is_remake', 'BOOLEAN', 1, None, 0)

(0, 'match_id', 'TEXT', 0, None, 1)
(1, 'side', 'TEXT', 1, None, 2)
(2, 'win', 'BOOLEAN', 1, None, 0)
(3, 'inhibitor_kills', 'INTEGER', 0, None, 0)
(4, 'bans', 'TEXT', 0, None, 0)
(5, 'tower_kills', 'INTEGER', 0, None, 0)
(6, 'baron_kills', 'INTEGER', 0, None, 0)
(7, 'dragon_kills', 'INTEGER', 0, None, 0)
(8, 'rift_herald_kills',

In [30]:
# Connect again and check tables
conn = sqlite3.connect('lol_data.db')
cursor = conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in database:", tables)

conn.close()

Tables in database: [('Accounts',), ('Matches',), ('Teams',), ('Participants',)]


In [46]:
# Displays table contents
conn = sqlite3.connect('lol_data.db')
cursor = conn.cursor()

def print_table_contents(db_path, table_name):
    try:
        # Connect to the SQLite database
        with sqlite3.connect(db_path) as conn:
            cursor = conn.cursor()
            
            # Execute SQL query to select all rows from the table
            cursor.execute(f"SELECT * FROM {table_name};")
            
            # Fetch and print all rows
            rows = cursor.fetchall()
            for row in rows:
                print(row)
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

db_path = 'lol_data.db'  # Replace with the path to your database file
table_name = 'Accounts'
print_table_contents(db_path, table_name)

('f3Zu9YSff-TwTwayEGzJFjLOo1PPVKE_Q57gtqUS22Rpdonk40pfTZhdAxiGfCuo8NPx4Mrbz-m-zQ', 'Evanlai', 'goma', 145, None, None, None, 'Diamond', 'I', 60)


In [32]:
# DROPS ALL TABLES
conn = sqlite3.connect('lol_data.db')
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS Accounts')
cursor.execute('DROP TABLE IF EXISTS Matches')
cursor.execute('DROP TABLE IF EXISTS Teams')
cursor.execute('DROP TABLE IF EXISTS Participants')

conn.commit()
conn.close()

In [20]:
conn.close()

In [33]:
conn = sqlite3.connect('lol_data.db')
cursor = conn.cursor()


cursor.execute('''
CREATE TABLE Accounts (
    account_id TEXT PRIMARY KEY,
    summoner_name TEXT NOT NULL,
    summoner_tagline TEXT NOT NULL,
    summoner_level INTEGER NOT NULL,
    summoner_flex_tier TEXT,
    summoner_flex_division TEXT,
    summoner_flex_lp INTEGER,
    summoner_soloq_tier TEXT,
    summoner_soloq_division TEXT,
    summoner_soloq_lp INTEGER
)
''')

cursor.execute('''
CREATE TABLE Matches (
    match_id TEXT PRIMARY KEY,
    patch TEXT NOT NULL,
    date TEXT NOT NULL,
    queue TEXT NOT NULL,
    duration TEXT NOT NULL,
    is_remake BOOLEAN NOT NULL
)
''')


cursor.execute('''
CREATE TABLE Teams (
    match_id TEXT,
    side TEXT NOT NULL,
    win BOOLEAN NOT NULL,
    inhibitor_kills INTEGER,
    bans TEXT,
    tower_kills INTEGER,
    baron_kills INTEGER,
    dragon_kills INTEGER,
    rift_herald_kills INTEGER,
    participants TEXT,
    PRIMARY KEY (match_id, side),
    FOREIGN KEY (match_id) REFERENCES Matches (match_id)
)
''')

cursor.execute('''
CREATE TABLE Participants (
    participant_id TEXT NOT NULL,
    account_id TEXT NOT NULL,
    match_id TEXT NOT NULL,
    summoner_name TEXT NOT NULL,
    side TEXT NOT NULL,
    lane TEXT NOT NULL,
    champion TEXT NOT NULL,
    runes TEXT NOT NULL,
    rune_stats TEXT NOT NULL,
    summoner_spell_d TEXT NOT NULL,
    summoner_spell_f TEXT NOT NULL,
    win BOOLEAN NOT NULL,
    kills INTEGER,
    deaths INTEGER,
    assists INTEGER,
    kda INTEGER,
    level INTEGER, 
    gold_earned INTEGER,
    total_minions_killed INTEGER,
    neutral_minions_killed INTEGER,
    items TEXT,
    total_damage_dealt INTEGER,
    total_damage_dealt_to_champions INTEGER,
    total_damage_taken INTEGER,
    damage_self_mitigated INTEGER,
    inhibitor_kills INTEGER,
    inhibitor_takedowns INTEGER,
    turret_kills INTEGER,
    turret_takedowns INTEGER,
    summoner_spell_1_casts INTEGER,
    summoner_spell_2_casts INTEGER,
    vision_score INTEGER,
    wards_placed INTEGER,
    vision_wards_placed INTEGER,
    wards_killed INTEGER,
    UNIQUE(participant_id),
    PRIMARY KEY (match_id, participant_id),
    FOREIGN KEY (match_id) REFERENCES Matches (match_id),
    FOREIGN KEY (match_id, side) REFERENCES Teams (match_id, side),
    FOREIGN KEY (account_id) REFERENCES Accounts (account_id)
)
''')

conn.commit()
conn.close()

In [39]:
'''
control flow: 
update player function that takes given account id
    if account id is in table, delete entry (let know user if adding new user or updating user)
    give account id to update account function
    go through player history
        if match isn't in sqlite database already, add match info to match table (let user know you are adding match of id x)
        add each team info to team table
        add each participant info to participant table


Riot API wording is different, but for our purposes, "Account" is more like just the LoL Summoner, and the "id" is the "puuid", 
which is more integral to identify the summoner rather than the Riot Account that the id and account_id's refer to.
'''

conn = sqlite3.connect('lol_data.db')
cursor = conn.cursor()

def update_account(acc):    
    summoner = acc.summoner
    flex_tier = None
    flex_division = None
    flex_lp = None
    soloq_tier = None
    soloq_division = None
    soloq_lp = None

    league_entries = summoner.league_entries
    for entry in league_entries: 
        if entry.queue == Queue.ranked_flex_fives:
            flex_tier = str(entry.tier)
            flex_division = str(entry.division)
            flex_lp = entry.league_points
        elif entry.queue == Queue.ranked_solo_fives:
            soloq_tier = str(entry.tier)
            soloq_division = str(entry.division)
            soloq_lp = entry.league_points

    cursor.execute('''
        INSERT OR IGNORE INTO Accounts (account_id, summoner_name, summoner_tagline, summoner_level, 
        summoner_flex_tier, summoner_flex_division, summoner_flex_lp,
        summoner_soloq_tier, summoner_soloq_division, summoner_soloq_lp)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (acc.puuid, acc.name, acc.tagline, summoner.level, 
         flex_tier, flex_division, flex_lp,
         soloq_tier, soloq_division, soloq_lp))

    conn.commit()
    
def add_match(match_id):
    print(match_id)
    # creates match object based on match id
    match = Match(id=match_id, region='NA')

    # we stop collecting data on this match (including on its team or its participants) if it is not a ranked soloq or flex game
    if(not (match.queue == Queue.ranked_flex_fives or match.queue == Queue.ranked_solo_fives)): return

    # adds match data into match table
    cursor.execute('''
        INSERT OR IGNORE INTO Matches (match_id, patch, date, queue, duration, is_remake)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (match_id,str(match.patch), datetime.fromisoformat(str(match.start)).strftime('%Y-%m-%d %H:%M:%S'), str(match.queue), 
          str(match.duration), match.is_remake))

    conn.commit()

    #routes to add_team() function to add teams to the table
    for team in match.teams:
        add_team(team, match_id)

def add_team(team, match_id):

    bans = ""
    for ban in team.bans:
        if ban is None: bans = bans + "None, "
        bans = bans + ban.name + ", " 

    barons = 0
    drakes = 0
    heralds = team.rift_herald_kills
    for player in team.participants:
        barons = barons + player.stats.baron_kills
        drakes = drakes + player.stats.dragon_kills

    participants = ""
    for player in team.participants:
        participants = participants + player.summoner.puuid
            
    cursor.execute('''
        INSERT OR IGNORE INTO Teams (match_id, side, win, inhibitor_kills, bans, tower_kills, baron_kills, dragon_kills, rift_herald_kills, participants)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (match_id, team.side, team.win, team.inhibitor_kills, bans, team.tower_kills, barons, drakes, heralds, participants))

    conn.commit()

    for participant in team.participants:
        add_participant(participant)

def add_participant(participant):
    print(participant.summoner_name)
   
def update_player(acc_id):
    summoner = Summoner(puuid = acc_id, region = 'NA')
    account = summoner.account
    
    cursor.execute("SELECT 1 FROM Accounts WHERE account_id = ?", (acc_id,))
    exists = cursor.fetchone()
    if exists:
        cursor.execute("DELETE FROM Accounts WHERE account_id = ?", (acc_id,))
        print(f"Updating account data for existing summoner {account.name}#{account.tagline}")
    else:
        print(f"Creating new account entry for summoner {account.name}#{account.tagline}")

    update_account(account)
    print()

    for match in summoner.match_history:
        cursor.execute("SELECT 1 FROM Matches WHERE match_id = ?", (match.id,))
        print(cursor.fetchone())
        exists = cursor.fetchone()  # fetchone() returns None if no result is found

        if exists:
            print(f"Match with id '{match.id}' exists in the Matches table.")
        else:
            print(f"No match found with id '{match.id}', now adding.")
            add_match(match.id)
        

update_player('f3Zu9YSff-TwTwayEGzJFjLOo1PPVKE_Q57gtqUS22Rpdonk40pfTZhdAxiGfCuo8NPx4Mrbz-m-zQ')
conn.close()

Updating account data for existing summoner Evanlai#goma
Master

Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/f3Zu9YSff-TwTwayEGzJFjLOo1PPVKE_Q57gtqUS22Rpdonk40pfTZhdAxiGfCuo8NPx4Mrbz-m-zQ/ids?start=0&count=100


ValueError: not enough values to unpack (expected 2, got 1)

In [53]:
print(len(mid.match_history))

Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/Zgzz5Ksm8xsEzFoGgTUWSA_M-Ejr4TnJxS9u4TiTE5KLG9-WgacmS4wu6UW4u4pHenV_kS6VZXxuSw/ids?start=0&count=100


ValueError: not enough values to unpack (expected 2, got 1)

In [34]:
conn.close()